### Se muestra el código completo con el cual se limpia la base original con una sóola función. 

In [291]:
import pandas as pd
import numpy as np
from scipy import stats 
import re
import string

In [122]:
data = pd.read_csv('C:/Users/Juan Carlos/Desktop/apuntes/Tweets.csv')

In [289]:
def suma_total(df):
    #This is our function that give us the final result.
    def Cambio_nombres (df):
        """This function recives a Data Frame and
        change the names of columns and their position 
        the Data Frame """
        #We make a copy of the Dataframe.
        data_modify = data.copy()
        #We change the names of columns.
        data_modify.columns = ['Tweet_id', 'Sentimiento_aerolinea', 'Grado_de_confianza',
                   'Razon_negativa', 'Confianza_razon_negativa', 'Aerolinea', 
                   'Aerolinea_sentimiento_oro', 'Nombre_usuario', 'Razon_negativa_oro', 
                   'Retweets', 'Comentario', 'Tweet_coord', 'Fecha_creación_tweet', 
                   'Región_tweet', 'Zona horaria']
        #We change the position of columns in the Dataframe, 
        order_columns = ['Tweet_id', 'Aerolinea', 'Nombre_usuario',
                  'Sentimiento_aerolinea', 'Grado_de_confianza',
                   'Razon_negativa', 'Confianza_razon_negativa',  
                   'Aerolinea_sentimiento_oro',  'Razon_negativa_oro', 
                   'Retweets', 'Comentario', 'Tweet_coord', 'Fecha_creación_tweet', 
                   'Región_tweet', 'Zona horaria']
        data_modify = data_modify[order_columns]
        return data_modify
    data_2 = Cambio_nombres(data)
    def Agregar_columnas(df):
        #We make a copy of the Dataframe thah we will use. 
        data_modify = data_2.copy()
        #We create an other function to apply the condition to each column of the Dataframe.
        def clasificacion (grado : str) -> str :
            """
            Receives a value and evaluates it to a string.
            Return
            INT """
            if grado <= .20:
                return 'Muy baja'
            elif grado <= .40:
                return 'Baja'
            elif grado <= .60:
                return 'Moderada'
            elif grado <=.80:
                return 'Alta'
            else:
                return 'Muy alta'
        #We add the two columns we need
        data_modify.insert(5,'Clasificacion_grado_de_confianza', data_modify['Grado_de_confianza'].apply(clasificacion))
        data_modify.insert(8,'Clasificacion_razon_negativa', data_modify['Confianza_razon_negativa'].apply(clasificacion))
        #We eliminate the columns that have more than 5500 nulls values.  
        nulls_cols = data_modify.isnull().sum()
        nulls_cols = nulls_cols[nulls_cols > 0].sort_values()
        cols_eliminate = list(nulls_cols[nulls_cols > 5500].index)
        data_cleaning = data_modify.drop(cols_eliminate, axis = 1)
        #In the columns that still have nulls values we writw the "None" caption. 
        data_cleaning[['Razon_negativa', 'Confianza_razon_negativa', 'Región_tweet', 'Zona horaria' ]] = data_cleaning[['Razon_negativa', 'Confianza_razon_negativa', 'Región_tweet', 'Zona horaria' ]].fillna('None')
        return(data_cleaning)
    Data_3 = Agregar_columnas(data_2)
    def Limpieza_texto(df): 
        #We create a copy of the Dataframe
        data_modify = Data_3.copy()
        #We change the index of the Dataframe. 
        data_cleaning = data_modify.set_index('Tweet_id')
        #We add a function that helpus to eliminate some caracters of the Comentario column.
        def clean_text(text):
            '''Make text uppercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
            text = text.upper()
            text = re.sub('\[.*?\]', '', text)
            text = re.sub('[% s]' % re.escape(string.punctuation), '', text)
            text = re.sub('\w*\d\w*', '', text)
            return text
        #We create a new clean column named "Comentario_limpio" with the help of Comentario column and is added in the tenth position.
        data_cleaning.insert(10,'Comentario_limpio', data_cleaning['Comentario'].apply(clean_text))
        #We eliminate the Comentario Column. 
        del(data_cleaning['Comentario'])
        return data_cleaning
    Data_4 = Limpieza_texto(Data_3)
    return(Data_4)
                

In [292]:
 Final_Dataframe =suma_total(data)

In [293]:
 Final_Dataframe 

,Aerolinea,Nombre_usuario,Sentimiento_aerolinea,Grado_de_confianza,Clasificacion_grado_de_confianza,Razon_negativa,Confianza_razon_negativa,Clasificacion_razon_negativa,Retweets,Comentario_limpio,Fecha_creación_tweet,Región_tweet,Zona horaria
Tweet_id,,,,,,,,,,,,,
570306133677760513,Virgin America,cairdin,neutral,1.0000,Muy alta,None,None,Muy alta,0,VIRGINAMERICA WHAT DHEPBURN SAID,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
570301130888122368,Virgin America,jnardino,positive,0.3486,Baja,None,0.0,Muy baja,0,VIRGINAMERICA PLUS YOUVE ADDED COMMERCIALS TO ...,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
570301083672813571,Virgin America,yvonnalynn,neutral,0.6837,Alta,None,None,Muy alta,0,VIRGINAMERICA I DIDNT TODAY MUST MEAN I NEED T...,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
570301031407624196,Virgin America,jnardino,negative,1.0000,Muy alta,Bad Flight,0.7033,Alta,0,VIRGINAMERICA ITS REALLY AGGRESSIVE TO BLAST O...,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
570300817074462722,Virgin America,jnardino,negative,1.0000,Muy alta,Can't Tell,1.0,Muy alta,0,VIRGINAMERICA AND ITS A REALLY BIG BAD THING A...,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
569587686496825344,American,KristenReenders,positive,0.3487,Baja,None,0.0,Muy baja,0,AMERICANAIR THANK YOU WE GOT ON A DIFFERENT FL...,2015-02-22 12:01:01 -0800,None,None
569587371693355008,American,itsropes,negative,1.0000,Muy alta,Customer Service Issue,1.0,Muy alta,0,AMERICANAIR LEAVING OVER MINUTES LATE FLIGHT ...,2015-02-22 11:59:46 -0800,Texas,None
569587242672398336,American,sanyabun,neutral,1.0000,Muy alta,None,None,Muy alta,0,AMERICANAIR PLEASE BRING AMERICAN AIRLINES TO,2015-02-22 11:59:15 -0800,"Nigeria,lagos",None


In [295]:
 Final_Dataframe.to_csv('Final_DataFrame.csv')